**Log file parsing**

We’re really not sure what happened, but we did manage to recover
the log file error.log. It seems to consist of a different log message
on each line. Each line begins with a character indicating the type of
log message it represents:

- ’I’ for informational messages,
- ’W’ for warnings, and
- ’E’ for errors.

The error message lines then have an integer indicating the severity
of the error, with 1 being the sort of error you might get around to
caring about sometime next summer, and 100 being epic, catastrophic
failure. All the types of log messages then have an integer time stamp
followed by textual content that runs to the end of the line. Here is a
snippet of the log file including an informational message followed
by a level 2 error message:

In [1]:
import Control.Applicative

data MessageType = Info
                 | Warning
                 | Error Int
  deriving (Show, Eq)

type TimeStamp = Int

data LogMessage = LogMessage MessageType TimeStamp String
                | Unknown String
  deriving (Show, Eq)

**Exercise 1** The first step is figuring out how to parse an individual
message. Define a function


    parseMessage :: String -> LogMessage


In [2]:
-- Exercise 1 The first step is figuring out how to parse an individual
-- message. Define a function
parseMessage :: String -> LogMessage
parseMessage str = case words str of
    ("I":ts:msg)      -> LogMessage Info (read ts :: Int) (unwords msg)
    ("W":ts:msg)      -> LogMessage Warning (read ts :: Int) (unwords msg)
    ("E":code:ts:msg) -> LogMessage (Error (read code :: Int)) (read ts :: Int) (unwords msg)
    str               -> Unknown (unwords str)

Putting the logs in order

Unfortunately, due to the error messages being generated by multiple
servers in multiple locations around the globe, a lightning storm, a
failed disk, and a bored yet incompetent programmer, the log messages
are horribly out of order. Until we do some organizing, there
will be no way to make sense of what went wrong! We’ve designed a
data structure that should help—a binary search tree of LogMessages:

In [3]:
data MessageTree = Leaf
                 | Node MessageTree LogMessage MessageTree
  deriving (Show, Eq)

**Exercise 2** Define a function

    insert :: LogMessage -> MessageTree -> MessageTree

which inserts a new LogMessage into an existing MessageTree, producing
a new MessageTree. insert may assume that it is given a
sorted MessageTree, and must produce a new sorted MessageTree
containing the new LogMessage in addition to the contents of the
original MessageTree.
However, note that if insert is given a LogMessage which is
Unknown, it should return the MessageTree unchanged.

In [4]:
insert :: LogMessage -> MessageTree -> MessageTree
insert (Unknown n) msgTree = msgTree
insert       logMsg (Leaf) = Node Leaf logMsg Leaf
insert msg@(LogMessage _ ts _) (Node left rootMsg@(LogMessage _ rootTs _) right) = case ts < rootTs of
    True  -> Node (insert msg left) rootMsg right
    False -> Node left rootMsg (insert msg right)

**Exercise 3** Once we can insert a single LogMessage into a MessageTree,
we can build a complete MessageTree from a list of messages. Specifi-
cally, define a function

    build :: [LogMessage] -> MessageTree

which builds up a MessageTree containing the messages in the list,
by successively inserting the messages into a MessageTree (beginning
with a Leaf).

In [5]:
build :: [LogMessage] -> MessageTree
build         [] = Leaf
build (msg:msgs) = insert msg (build msgs)

Exercise 4 Finally, define the function

inOrder :: MessageTree -> [LogMessage]

which takes a sorted MessageTree and produces a list of all the
LogMessages it contains, sorted by timestamp from smallest to biggest.
(This is known as an in-order traversal of the MessageTree.)
With these functions, we can now remove Unknown messages and
sort the well-formed messages using an expression such as:

inOrder (build tree)

[Note: there are much better ways to sort a list; this is just an exercise
to get you working with recursive data structures!]

In [24]:
inOrder :: MessageTree -> [LogMessage]
inOrder Leaf = []
inOrder (Node left logMsg right) = inOrder left ++ [logMsg] ++ inOrder right

**Exercise 5** Now that we can sort the log messages, the only thing
left to do is extract the relevant information. We have decided that
“relevant” means “errors with a severity of at least 50”.
Write a function

    whatWentWrong :: [LogMessage] -> [String]

which takes an unsorted list of LogMessages, and returns a list of the
messages corresponding to any errors with a severity of 50 or greater,
sorted by timestamp. (Of course, you can use your functions from the
previous exercises to do the sorting.)

In [171]:
messages = ["I 6 Completed armadillo processing", "I 1 Nothing to report", "E 99 10 Flange failed!", "I 4 Everything normal", "I 11 Initiating self-destruct sequence", "E 70 3 Way too many pickles", "E 65 8 Bad pickle-flange interaction detected", "W 5 Flange is due for a check-up", "I 7 Out for lunch, back in two time steps", "E 20 2 Too many pickles", "I 9 Back from lunch"]

In [172]:
parseToMessages :: [String] -> [LogMessage]
parseToMessages [] = []
parseToMessages (str:strs) = parseMessage str : parseToMessages strs

In [170]:
whatWentWrong :: [LogMessage] -> [String]
whatWentWrong logs = case (sever . inOrder . build) logs of
    [] -> []
    ((LogMessage _ _ str):rest) -> str : whatWentWrong rest

<interactive>:3:5: error:
    • Couldn't match expected type ‘Int -> [LogMessage]’ with actual type ‘[t0]’
    • In the pattern: []
      In a case alternative: [] -> []
      In the expression:
        case (sever . inOrder . build) logs of {
          [] -> []
          ((LogMessage _ _ str) : rest) -> str : whatWentWrong rest }
<interactive>:4:6: error:
    • Couldn't match expected type ‘Int -> [LogMessage]’ with actual type ‘[LogMessage]’
    • In the pattern: (LogMessage _ _ str) : rest
      In a case alternative: ((LogMessage _ _ str) : rest) -> str : whatWentWrong rest
      In the expression:
        case (sever . inOrder . build) logs of {
          [] -> []
          ((LogMessage _ _ str) : rest) -> str : whatWentWrong rest }

In [157]:
sever :: [LogMessage] -> Int -> [LogMessage]
sever (logMsg@(LogMessage (Error code) _ _):logs) n = case code <= n of
    True  -> logMsg : sever logs n
    False -> []
sever _ _ = []

In [166]:
sever [LogMessage (Error 70) 3 "Way too many pickles"] 5

[]

In [176]:
inOrder $ build $ parseToMessages messages

[LogMessage Info 1 "Nothing to report",LogMessage (Error 20) 2 "Too many pickles",LogMessage (Error 70) 3 "Way too many pickles",LogMessage Info 4 "Everything normal",LogMessage Warning 5 "Flange is due for a check-up",LogMessage Info 6 "Completed armadillo processing",LogMessage Info 7 "Out for lunch, back in two time steps",LogMessage (Error 65) 8 "Bad pickle-flange interaction detected",LogMessage Info 9 "Back from lunch",LogMessage (Error 99) 10 "Flange failed!",LogMessage Info 11 "Initiating self-destruct sequence"]

<interactive>:1:14: error:
    • Couldn't match type ‘String’ with ‘Char’
      Expected type: String
        Actual type: [String]
    • In the first argument of ‘parseMessage’, namely ‘messages’
      In the expression: parseMessage messages
      In an equation for ‘it’: it = parseMessage messages